# Whizzle model v5 retrain - New Member

In [1]:
import os
import pandas as pd
import numpy as np
from time import time
from tqdm.notebook import tqdm
import random
import pickle

from scipy.sparse import coo_matrix, csr_matrix

from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k, reciprocal_rank

from hyperopt import fmin, hp, tpe, Trials

from sklearn.base import clone

## Normalization
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import seaborn as sns
import matplotlib.pyplot as plt

from dataclasses import dataclass

C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
item_features = pd.read_csv("dataset/item_features.csv", index_col=0, encoding="UTF-8")
user_features = pd.read_csv("dataset/user_features.csv", index_col=0, encoding="UTF-8")
train_rating = pd.read_csv("dataset/train_rating.csv", index_col=0, encoding="UTF-8")
test_rating = pd.read_csv("dataset/test_rating.csv", index_col=0, encoding="UTF-8")
whisky = pd.read_csv("dataset/whisky.csv", index_col=0, encoding="UTF-8")

In [12]:
def make_source(data):
    source = []
    for row in data.itertuples(index=False):
        meta = {feat: value for feat, value in zip(data.columns[1:], row[1:])}
        source.append((row[0], meta))
    return source

## 신규 사용자 입력 가정
---
- 저장된 모델날짜 이후로 변경된 모든 데이터를 전달한다.
    - 학습에 포함된 사용자의 rating, preference는 전달하지 않아도 된다.
- 즉, 사용자 기준으로 모델에 포함되지 않은 사용자의 rating, preference 데이터를 전달받는다

## 입력 기준
---
content : {
    users: 마지막 6
    ratings: [
        {
            member_id: 1,
            whiskiy_id: 515,
            rating: 10,
        },
        {
            member_id: 2,
            whiskiy_id: 443,
            rating: 10,
        },
        {
            member_id: 3,
            whiskiy_id: 515,
            rating: 8,
        },
        {
            member_id: 4,
            whiskiy_id: 111,
            rating: 9,
        }
    ],
    preferences: [
        {
            member_id: 1,
            smoky: 0.51,
            ...
        },
        {
            member_id: 1,
            smoky: 0.51,
            ...
        },
        {
            member_id: 1,
            smoky: 0.51,
            ...
        }
    ]
}

In [ ]:
## 신규 사용자 rating df 만들기
def make_new_rating_df(ratings):
    return pd.DataFrame([(rating.user_id+119515, rating.whisky_id, rating.rating) for rating in ratings], columns=['user_id', 'whisky_id', 'rating'])
